In [0]:
%run "/Users/sivajujare@outlook.com/utilities"


In [0]:
filename = dbutils.widgets.get('a_filename')
fname = filename.split('.')[0]
print(fname)

In [0]:
import datetime
from pyspark.sql.functions import lit
dates = datetime.datetime.now().strftime("%Y-%m-%d")

In [0]:
orders_df = read_df('csv','/mnt/retailadls218/raw/orders.csv') \
                .withColumn("ingest_date", lit(dates))


In [0]:
from pyspark.sql.functions import col, count

dups = orders_df.select('order_id') \
                .groupBy('order_id') \
                .agg(count('order_id').alias('dcount')) \
                .filter(col('dcount') > 1) 

dups_order_ids = [row.order_id for row in dups.select("order_id").distinct().collect()]

dups_orders = orders_df.filter(col("order_id").isin(dups_order_ids))

In [0]:
if (dups.rdd.isEmpty() == False):
    write_w_part(dups_orders, "parquet", "/mnt/retailadls218/rejects/orders/", "ingest_date", "append", "retail.orders_rejects")
    dbutils.notebook.exit("duplicates found, so exiting the process")

In [0]:
databricksScope='retail-scope'

dbServer= dbutils.secrets.get(scope=databricksScope,key='sql-server-name') 
dbPort='1433'
dbName= dbutils.secrets.get(scope=databricksScope,key='sql-database-name') 
dbUser= dbutils.secrets.get(scope=databricksScope,key='sql-user') 
dbPassword=dbutils.secrets.get(scope=databricksScope,key='sql-password')

lkp_df = read_sql_table(dbServer, dbPort, dbName, dbUser, dbPassword, 'dbo.valid_order_status')

In [0]:
status_names = [row.status_name for row in lkp_df.select("status_name").distinct().collect()]

not_mat_df = orders_df.filter(~col("order_status").isin(status_names))

In [0]:
if (not_mat_df.rdd.isEmpty() == False):
    write_w_part(not_mat_df, "parquet", "/mnt/retailadls218/rejects/orders/", "ingest_date", "append", "retail.orders_rejects")
    dbutils.notebook.exit("missing records found, so exiting the process")

In [0]:
missing_status_ids = [row.order_id for row in not_mat_df.select("order_id").distinct().collect()]
dup_ord_ids = [row.order_id for row in dups.select("order_id").distinct().collect()]

orders_wo_missing_status_ids = orders_df.filter(~col("order_id").isin(missing_status_ids))
orders_wo_dups = orders_wo_missing_status_ids.filter(~col("order_id").isin(dup_ord_ids))

display(orders_wo_dups)

order_id,order_date,customer_id,order_status,ingest_date
2222222,00:00.0,256,PENDING_PAYMENT,2024-03-01


In [0]:
write_df(orders_wo_dups, "parquet", "/mnt/retailadls218/staging/orders/", "overwrite")

DataFrame[order_id: int, order_date: string, customer_id: int, order_status: string, ingest_date: string]